### 어노테이션 파일의 라벨링 구조를 확인하고 화면에 바운딩 박스를 그려 실제로 확인

원천 데이터 파일은 D드라이브에 아래와 같이 저장되어 있습니다.

>&nbsp;&nbsp;<br>
>D드라이브<br>
>&nbsp;&nbsp;└ pgu_traffic_dataset<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ sources<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ images → 이미지 파일(png) 18,481개<br>
>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└ annotations → 라벨 파일(json) 18,481개<br>
>&nbsp;&nbsp;

In [ ]:
import os
import cv2
import json
import random

json 문자열로 되어 있는 어노테이션 파일을 로드한 후 dict로 변환하고 구조와 내용을 확인해 봅니다.

In [ ]:
img_path = "D:/pgu_traffic_dataset/sources/images" #이미지 파일이 있는 폴더 경로를 변수에 저장
ann_path = "D:/pgu_traffic_dataset/sources/annotations" #어노테이션 파일이 있는 폴더 경로를 변수에 저장
img_file_list = os.listdir(img_path) #이미지 폴더에 있는 파일들의 목록을 불러와 변수에 저장
ann_file_list = os.listdir(ann_path) #어노테이션 폴더에 있는 파일들의 목록을 불러와 변수에 저장

In [ ]:
#가장 첫 번째 어노테이션 파일을 불러와 내용 확인
ann_sample_str = open("{}/{}".format(ann_path, ann_file_list[0]), "r").read()
ann_sample = json.loads(ann_sample_str) #json 유형으로 저장되어 있음. json 모듈을 이용해 dict로 변환
ann_sample["info"] # -> 촬영에 대한 기본 정보(촬영일시, 촬영자 등)를 담고 있음

In [ ]:
ann_sample["images"] # -> 이미지 파일에 대한 기본 정보(파일명, 해상도)를 담고 있음

In [ ]:
ann_sample["annotations"] # -> 이미지 파일에 대응되는 라벨링 정보(바운딩박스 좌표 등)를 담고 있음

In [ ]:
len(ann_sample["annotations"]) # -> 28개의 바운딩 박스 정보(딕셔너리)로 구성된 리스트. 즉, 이 경우 한장의 사진에 28개의 라벨이 있음

In [ ]:
ann_sample["annotations"][0] # -> 0번째 라벨 정보를 보면 바운딩 박스 정보와 다른 정보들을 담고 있음을 알 수 있음

openCV를 활용해 이미지 파일을 화면에 띄운 다음 라벨링 정보를 반영해 바운딩박스를 그려봅니다.

In [ ]:
#openCV imread를 통해 ann_sample에 지정된 이미지 파일을 numpy ndarray로 로드
src = cv2.imread("{}/{}".format(img_path, ann_sample["images"]["file_name"]))
dst = src.copy() #바운딩박스를 그릴 이미지를 원본에서 복사해 dst 변수에 저장
for bbox in ann_sample["annotations"]: #이 이미지에 지정된 라벨링 개수만큼 반복
    x = int(bbox["bbox"][0]) #바운딩 박스의 좌상단 x 좌표
    y = int(bbox["bbox"][1]) #바운딩 박스의 좌상단 y 좌표
    w = int(bbox["bbox"][2]) #바운딩 박스의 너비
    h = int(bbox["bbox"][3]) #바운딩 박스의 높이
    c_no = bbox["category_id"] #클래스 넘버 : 1 → 기둥(전신주 등), 2 → 도로표지판, 3 → 신호등 ...
    dst = cv2.rectangle(dst, (x, y, w, h), (0, 255, 0), 1)
    dst = cv2.putText(dst, str(c_no), (x, y), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 0), 1)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

반복문을 통해 랜덤으로 이미지를 불러와 라벨링이 제대로 되어 있는지 확인합니다.

In [ ]:
#랜덤값을 인덱스로 하여 총 30장의 이미지만 무작위로 뽑아올 수 있게 함
random_no_list = [random.randrange(0, len(ann_file_list)) for _ in range(0, 30)]
for idx in random_no_list:
    ann_str = open("{}/{}".format(ann_path, ann_file_list[idx]), "r").read()
    ann = json.loads(ann_str)
    #openCV를 활용해 이미지 파일을 띄운 다음 라벨링 정보를 반영해 바운딩박스를 그려봄
    src = cv2.imread("{}/{}".format(img_path, ann["images"]["file_name"]))
    dst = src.copy()
    for bbox in ann["annotations"]:
        x = int(bbox["bbox"][0]) #바운딩 박스의 좌상단 x 좌표
        y = int(bbox["bbox"][1]) #바운딩 박스의 좌상단 y 좌표
        w = int(bbox["bbox"][2]) #바운딩 박스의 너비
        h = int(bbox["bbox"][3]) #바운딩 박스의 높이
        c_no = bbox["category_id"] #클래스 넘버
        # 1 → 기둥(전신주 등)
        # 2 → 도로표지판
        # 3 → 신호등
        # 4 → 터널 천장에 있는 환풍기
        # 5 → 터널 벽에 있는 조명
        # 6 → 터널 벽에 있는 표지판
        if c_no > 1:
            dst = cv2.rectangle(dst, (x, y, w, h), (0, 255, 0), 1)
            dst = cv2.putText(dst, str(c_no), (x, y), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0, 255, 0), 1)
    cv2.imshow("dst", dst)
    if cv2.waitKey() == 27:
        break
    else:
        continue
cv2.destroyAllWindows()